In [0]:
import pandas as pd
import re
import numpy
import os
###!!! Attention!!! Для корректной работы jiant необходима версия allennlp===0.9.0
### с более поздними версиями могут возникнуть проблемы в совместимости
!pip install transformers allennlp===0.9.0
#!pip install jsondiff
#!pip install pyhocon
#!pip install python-Levenshtein

**Пример запуска baseline модели для Russian-SUPERGlue**



In [0]:
#!unzip jiant-russian.zip
os.chdir('./jiant-russian/')

**Настройка config'а**


Для работы pipelin'а необходимо указать путь к данным и путь для сохранения модели в  `user_config.sh`:

1) `export JIANT_DATA_DIR=./combined/` - путь к данным для тасков. По умолчанию указана папка `combined`, которую необходимо поместить в корневую директорию.

2) `export JIANT_PROJECT_PREFIX=./model_dir/` - путь, где сохраняются модели, логи, результаты предсказания. По умолчанию указана `./model_dir/`.

Параметры, специфичные для модели и параметры обучения указываются в конфиге модели: `jiant/config/superglue_bert.conf`. 


Например, там задается:

 `input_module` - используемая бертовская модель (по умолчанию `"DeepPavlov/rubert-base-cased-conversational"`)

 `exp_name` - короткое название модели, по умолчанию `rubert`

 `max_val, max_epochs, learning_rate` и другие.




**Данные** 

Данные можно скачать с сайта [RussianGLUE](https://russiansuperglue.com/tasks/). По [ссылке](https://russiansuperglue.com/tasks/download) можно скачать данные для всех таксков разом. Дальше достаточно будет разархивировать архив и указать путь к нему в качестве `export JIANT_PROJECT_PREFIX=` в конфиге.

**Структура данных**

Данные для каждого из заданий должны лежать в подпапке с аналогичным названием в `export JIANT_PROJECT_PREFIX=` (в [общем архиве с данными](https://russiansuperglue.com/tasks/download) как раз такая струтура): 

1) DaNetQa

2) LiDiRus

3) MuSeRC

4) PARus

5) RCB

6) RuCoS

7) RUSSE

8) RWSD

9) TERRa

Для запуска всех тасков кроме диагностического LiDiRus в папке должны находиться `train.jsonl, val.jsonl, test.jsonl`. Для LiDiRus в соответствующую папку надо положить `LiDiRus.jsonl`.



**Запуск baseline**

Запуска baselin'ов осуществляется с помощью скрипта `./scripts/russian-superglue-baselines.sh`. Для обучения модели и получения предсказаний необходимо запустить скрипт с указанием необходимого таска (предсказания для диагностического `LiDiRus` вычисляются при обучении на TERRa, то есть при запуске terra модель делает предсказания как для terra_test, так и для lidirus.


**Usage:**
```
   ./scripts/superglue-baselines.sh ${TASK} ${GPU_ID} ${SEED}
   - TASK: one of {"danetqa", "rcb", "parus", "muserc", "rucos", "terra", "russe", "rwsd", "all"},

   - GPU_ID: GPU to use, or -1 for CPU. Defaults to 0.
   - SEED: random seed. Defaults to 111.
```

Для запуска всех тасков необходимо выбрать опцию `all.` При этом таски запускаются в следующем порядке (`LidiRus` вызывается внутри `TERRa`):     
```    
    rwsd
    russe
    terra
    rucos
    muserc
    parus
    rcb
    danetqa
```

Ниже приведен пример базового запуска для задания `TERRa`.

In [0]:
!chmod 755 ./scripts/russian-superglue-baselines.sh 
! ./scripts/russian-superglue-baselines.sh "terra"

**Results**

Результаты сохраняются в `model_dir/exp_name/task_name`. 

Скоры модели на валидации по всем отработанным таскам записываются в `model_dir/exp_name/results.tsv`. 


In [17]:
#TERRa results
print(open('./model_dir/rubert/results.tsv').readlines()[-1])

terra	micro_avg: 0.658, macro_avg: 0.658, terra_accuracy: 0.658



In [18]:
#LiDiRus results
print(open('./model_dir/rubert/results.tsv').readlines()[0])

terra	micro_avg: 0.000, macro_avg: 0.000, lidirus_lex_sem: -0.067, lidirus_lex_sem__Redundancy: 0.060, lidirus_lex_sem__Symmetry/Collectivity: 0.175, lidirus_lex_sem__Named entities: -0.169, lidirus_lex_sem__Lexical entailment;Quantifiers: 0.000, lidirus_lex_sem__Lexical entailment: -0.030, lidirus_lex_sem__Quantifiers: -0.219, lidirus_lex_sem__Lexical entailment;Factivity: 0.000, lidirus_lex_sem__Factivity;Quantifiers: 0.000, lidirus_lex_sem__Factivity: 0.043, lidirus_pr_ar_str: -0.152, lidirus_pr_ar_str__Intersectivity;Ellipsis/Implicits: 0.000, lidirus_pr_ar_str__Coordination scope;Prepositional phrases: 0.000, lidirus_pr_ar_str__Coordination scope: 0.000, lidirus_pr_ar_str__Active/Passive;Prepositional phrases: 0.000, lidirus_pr_ar_str__Core args: 0.065, lidirus_pr_ar_str__Active/Passive: -0.333, lidirus_pr_ar_str__Restrictivity;Anaphora/Coreference: 0.000, lidirus_pr_ar_str__Genitives/Partitives: 0.000, lidirus_pr_ar_str__Relative clauses;Restrictivity: 0.000, lidirus_pr_ar_str__A